In [1]:
import os
import sys
sys.path.append("/home/dux/")
sys.path.append("/home/dux/surface_sampling/sgmc_surf")

from mcmc import initialize_slab, spin_flip, change_site, get_random_idx
from catkit.gen.adsorption import get_adsorption_sites

from ase.calculators.eam import EAM
from ase.calculators.lammpsrun import LAMMPS

import numpy as np

In [14]:
# Cu alat from https://www.copper.org/resources/properties/atomic_properties.html
Cu_alat = 3.6147
slab = initialize_slab(Cu_alat)

# get ALL the adsorption sites
# top should have connectivity 1, bridge should be 2 and hollow more like 4
coords, connectivity, sym_idx = get_adsorption_sites(slab, symmetry_reduced=False)
print(f"In pristine slab, there are a total of {len(connectivity)} sites")

# state of each vacancy in slab. for state > 0, it's filled, and that's the index of the adsorbate atom in slab
state = np.zeros(len(coords), dtype=int)

temp = 300
pot = 2

In pristine slab, there are a total of 64 sites


In [15]:
# test top
top_site = get_random_idx(connectivity, type="top")
bridge_site = get_random_idx(connectivity, type="bridge")
hollow_site = get_random_idx(connectivity, type="hollow")

# add to 3 sites
spin_flip(state, slab, temp, pot, coords, connectivity, save_cif=True, iter=1, site_idx=top_site, testing=True, folder_name="different_sites_noalat")
spin_flip(state, slab, temp, pot, coords, connectivity, save_cif=True, iter=2, site_idx=bridge_site, testing=True, folder_name="different_sites_noalat")
spin_flip(state, slab, temp, pot, coords, connectivity, save_cif=True, iter=3, site_idx=hollow_site, testing=True, folder_name="different_sites_noalat")

DEBUG:
 we are at iter 1
DEBUG:idx is 11 with connectivity 1 at [ 7.66793665  5.11195776 20.42205   ]
DEBUG:before proposed state is
DEBUG:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
DEBUG:chosen site is empty
DEBUG:adsorbing from empty to adsorbate Cu and potential 2
DEBUG:current slab has 64 atoms
DEBUG:proposed slab has 65 atoms
DEBUG:after proposed state is
DEBUG:[ 0  0  0  0  0  0  0  0  0  0  0 64  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
DEBUG:
 we are at iter 2
DEBUG:idx is 30 with connectivity 2 at [ 5.11195776  8.94592609 20.42205   ]
DEBUG:before proposed state is
DEBUG:[ 0  0  0  0  0  0  0  0  0  0  0 64  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
DEBUG:chosen

(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 64,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 65,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 66,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
 Gratoms(symbols='Cu67', pbc=[True, True, False], cell=[10.223915527820054, 10.223915527820054, 35.42204999999999], initial_magmoms=..., surface_atoms=..., tags=...),
 0,
 False)

In [16]:
# Test cases
import logging
logging.basicConfig(
    format="%(levelname)s:%(message)s",
    level=logging.DEBUG,
    datefmt="%m/%d/%Y %I:%M:%S %p",
)

site_idx = top_site
# start_ads = slab[state[site_idx]].symbol
# print(f"start ads {start_ads}")
adsorbates = ['In', 'Cu', 'Ga']
pots = [2, 3, 5]

# try top already filled site
slab, state, delta_pot, start_ads, end_ads = change_site(slab, state, pots, adsorbates, coords, site_idx)
print("Trial move")
print(f"slab={slab},state={state},delta_pot={delta_pot},start_ads={start_ads},end_ads={end_ads}")

slab, state, delta_pot, start_ads, end_ads = change_site(slab, state, pots, adsorbates, coords, site_idx, start_ads=end_ads, end_ads=start_ads)
print("Revert move")
print(f"slab={slab},state={state},delta_pot={delta_pot},start_ads={start_ads},end_ads={end_ads}")
# ads_pot_dict = dict(zip(adsorbates, pots))

# ads_choices = adsorbates.copy()
# ads_choices.remove(start_ads)
# print(f"ads choices are {ads_choices}")

# print(f"{ad.remove("")}")

DEBUG:chosen site already adsorbed
DEBUG:chosen ads is None
DEBUG:current slab has 67 atoms
DEBUG:desorbing Cu
DEBUG:proposed slab has 66 atoms
DEBUG:chosen site is empty
DEBUG:adsorbing from empty to adsorbate Cu and potential 3
DEBUG:current slab has 66 atoms
DEBUG:proposed slab has 67 atoms


Trial move
slab=Gratoms(symbols='Cu66', pbc=[True, True, False], cell=[10.223915527820054, 10.223915527820054, 35.42204999999999], initial_magmoms=..., surface_atoms=..., tags=...),state=[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0 64  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0 65  0  0  0  0  0  0  0  0  0  0  0  0  0  0],delta_pot=-3,start_ads=Cu,end_ads=None
Revert move
slab=Gratoms(symbols='Cu67', pbc=[True, True, False], cell=[10.223915527820054, 10.223915527820054, 35.42204999999999], initial_magmoms=..., surface_atoms=..., tags=...),state=[ 0  0  0  0  0  0  0  0  0  0  0 66  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0 64  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0 65  0  0  0  0  0  0  0  0  0  0  0  0  0  0],delta_pot=3,start_ads=None,end_ads=Cu


In [6]:
# Possible test cases, can go in sequence
# 1) empty list
# 2) start with ads, go to empty
# 3) start wtih ads, go to another ads

In [ ]:
# Other test cases for spin flip

In [17]:
delta_pot

3